# <center> R4.04 Méthodes d'optimisation <br> $\mathbf{SAT}$-Solvers </center>
<center> 2023/2024 - Tom Ferragut, Thibault Godin & Lucie Naert </center>
<center> IUT de Vannes, BUT Informatique </center>


In [3]:
import numpy as np 
import pycosat
import itertools
import sys, getopt 
import time
from pprint import pprint

#pip install pycosat

*********

# $\mathbf{ SAT}$


En première année, vous avez vu les concepts d'assertions et de formules (booléennes) logiques. Par exemple la formule

$$A :=  (\neg P \wedge Q) \vee  (P\wedge \neg Q) $$


qui a pour table de vérité <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) :


|P|Q|A|
| ------------- |:-------------:| -----:|
|V|V|F|
|V|F|V|
|F|V|V|
|F|F|F|


Étant donnée une assertion $A$, on dira que l'assertion est _satisfaisable_ (satisfiable) s'il existe des valeurs de vérités pour les variables atomiques (ici $P$ et $Q$) telle que $A$ soit vraie. Dans notre exemple $A$ est satisfaisable, et un _témoin_ (witness) est le choix de valeurs $P=V$ et $Q=F$. 


La question _"étant donnée une assertion logique $A$, déterminer si elle est satisfaisable"_ est un problème majeur en informatique, appelé [SAT](https://fr.wikipedia.org/wiki/Probl%C3%A8me_SAT)


<a name="cite_note-1"></a>1. [^](#cite_ref-1) On appelle généralement cet opérateur xor (ou exclusif), noté $\oplus$



**_question 1_**
Déterminer si les assertions suivantes sont satisfaisables, et donner un témoin si elle le sont.


- $A_1 :=  \neg (P \wedge Q) \vee R $
- $A_2 :=  (P \Longleftrightarrow Q) \wedge (P \oplus Q) $
- $A_3 :=  \neg (P \wedge Q) \vee P $
- $A_4 :=  \neg (P \wedge Q \wedge \neg R) \vee (P \wedge R)$


Réponse : 


*********

## CNF

Pour les problèmes de satisfaisabilité, on a généralement un grand nombre de variables booléennes. On les notera généralement $x_1, x_2, \dots, x_n$, et on dira que $n$ est la _taille_ de l'instance.
De plus, on a vu en R1.06 que toute formule peut s'écrire à l'aide des seules opérations $\neg, \vee, \wedge$, et l'on peut même exiger que les $\neg$ ne porte que sur des propositions atomiques (les $x_i$).


**_question 2_**

Transformer les propositions suivantes de manière à n'utiliser que les opérations $\neg, \vee, \wedge$, et que les $\neg$ ne porte que sur des propositions atomiques.
- $A_1 :=  \neg (P \wedge Q) \vee P $
- $A_1 :=  \neg (P \wedge Q \wedge \neg R) \vee (P \wedge R)$
- $A_3 := \neg (P \Longrightarrow Q)$
- $A_4 :=  (P \Longleftrightarrow Q) \vee \neg P $




Réponse : 



Pour faciliter la saisie des assertion, on va normaliser un peu plus encore : on va demander à ce que nos formules soient des conjonctions (des "et") de disjonctions (des "ou") de variables. On appelle cette forme [CNF](https://fr.wikipedia.org/wiki/Forme_normale_conjonctive)<a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2).

Exemples d'assertions en CNF : $(x_1 \vee x_2 \vee \neg x_3) \wedge (x_1 \vee x_3)$ et  $(x_1 \vee \neg x_5 \vee x_4) \wedge (\neg x_1 \vee  x_5 \vee x_3 \vee x_4) \wedge (\neg x_3 \vee \neg x_4)$.

Les exemples suivants ne sont pas en CNF : $ \neg(x_1 \wedge x_2 \wedge \neg x_3) \vee (x_1 \wedge x_3)$ et $(x_1 \wedge \neg x_5 \wedge x_4) \vee  (\neg x_1 \wedge  x_5 \wedge x_3 \wedge x_4) \vee  (\neg x_3 \wedge \neg x_4)$ (le dernier exemple est en forme normale disjonctive, DNF)


On appelera _clause_ les parties entre deux $\wedge$'s consécutifs.

<a name="cite_note-2"></a> 2. [^](#cite_ref-2) On peut aller encore plus loin en imposant que la formule soit une conjonction (un "et") de disjonctions (des "ou") d'au plus $3$ variables. On optient le problème [3-SAT](https://fr.wikipedia.org/wiki/Probl%C3%A8me_3-SAT) qui est équivalent au problème $\mathbf{SAT}$

*********
**_question 3_**

Mettre $A_3$ et $A_4$ en CNF.

Réponse : 



*********
Une manière d'encoder une formule booléenne est la suivante : chaque clause est encodée dans un tableau d'entiers, la variable $x_5$ étant représentée par <tt>5</tt> et la variable $\neg x_5$ étant représentée par <tt>-5</tt>


Par exemple :

In [7]:
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]

représente la formule 


 $(x_1 \vee \neg x_5 \vee x_4) \wedge (\neg x_1 \vee  x_5 \vee x_3 \vee x_4) \wedge (\neg x_3 \vee \neg x_4)$ 
 
 
Ici, on a 5 variables et 3 clauses, la première clause étant $(x_1 \vee \neg x_5 \vee x_4)$. 

*********


On définit une fonction pour afficher des CNF et une fonction retournant le nombre de variables et de clauses d'une formule en CNF

In [8]:
def display(cnf):
    monCnf = "("
    for i in range(len(cnf)):
        for j in range(len(cnf[i])):
            if cnf[i][j] < 0:
                monCnf += "non "
            monCnf += "x"+str(abs(cnf[i][j]))
            if j != len(cnf[i]) - 1:
                monCnf +=" ou "
        if i != len(cnf) -1 :
            monCnf += ") et ("
        else :
            monCnf += ")"
    return monCnf

strcnf = display(cnf)
print(strcnf)


def cnflen(cnf):
    #returns the number of variables and of clauses of a boolean formula given under CNF
    return max([abs(max(max(cnf, key=max))),abs(min(min(cnf, key=min)))]),len(cnf)

nbVar, nbClause = cnflen(cnf)

print("Nombre de variable : ",nbVar, "\nNombre de clauses : ", nbClause)



(x1 ou non x5 ou x4) et (non x1 ou x5 ou x3 ou x4) et (non x3 ou non x4)
Nombre de variable :  5 
Nombre de clauses :  3


*********
**_question 4_**


Essayer de résoudre quelques instances en CNF sur [SAT Game](http://www.cril.univ-artois.fr/~roussel/satgame/satgame.php?level=2&lang=fr)


*********

# $\mathbf{SAT}$-Solver



La résolution algorithmique d'un problème $\mathbf{SAT}$ est assez facile : on peut tester tous les cas possibles, jusqu'à ce que l'on trouve un témoin ou que l'on ait essayé toutes les combinaisons. S'il y a $n$ variables, il n'y a "que" $2^n$ cas à tester.


*********

**_question 5_**


1. Écrire une fonction `eval(cnf,wit)` qui prend en entrée `cnf` une formule booléenne en CNF à $n$ variables et `wit` un tableau de $n$ valeurs booléennes (0 ou 1) et retourne l'évaluation de la formule sur ces valeurs 
2. Faire 3 tests : 
    - un test avec un premier CNF et un ensemble de valeurs pour lesquelles l'évaluation est "True"
    - un test avec le même CNF et un ensemble de valeurs pour lesquelles l'évaluation est "False"
    - un test avec un autre CNF

In [17]:
def eval(cnf,wit):
    
   
        
    return "todo"

#Exemple True             
wit=[1, 0, 0, 0, 1]               
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]
print(eval(cnf,wit))

True
False
(x1) et (x2)
(2, 2)
False


*********

**_question 6_**


Écrire une fonction `bruteSAT(cnf, verbose = False)` qui essaie de resoudre une formule en CNF par force-brute (en essayant toutes les possibilités) et renvoie un témoin si la formule est satisfaisable et -1 sinon. Si `verbose = True` et qu'un témoin est trouvé, le nombre d'essai avant de trouver un témoin doit être affiché ainsi que le nombre maximum d'essais possibles.

_On pourra utiliser le module itertool.product de python_


In [18]:
def bruteSAT(cnf, verbose = False):
    return "todo"
    
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]

print(display(cnf))
print(bruteSAT(cnf, True))
print(bruteSAT([[1,2]], True))
print(bruteSAT([[1],[2]]))
print(bruteSAT([[1,2],[-1,2],[1,-2]]))
print(bruteSAT([[1,2],[-1,2],[1,-2],[-1,-2]]))




(x1 ou non x5 ou x4) et (non x1 ou x5 ou x3 ou x4) et (non x3 ou non x4)
Trouvé en 1 essais sur 32
[0, 0, 0, 0, 0]
Trouvé en 2 essais sur 4
[0, 1]
[1, 1]
[1, 1]
-1


*********

## PycoSAT

On ne sait pas aujourd'hui s'il existe un algorithme polynômial (_ie_ s'executant en temps raisonnable) pour résoudre $\mathbf{SAT}$. Cependant ce problème, bien qu'abstrait et théorique, est central en informatique ;  et on peut résoudre beaucoup de problèmes (qui paraissent pourtant complétement différents) en les traduisant en des formules logiques.
On verra partie 2 l'exemple du Sudoku, mais bien d'autres [problèmes](https://fr.wikipedia.org/wiki/Liste_de_probl%C3%A8mes_NP-complets)  (3-coloration, emplois du temps ...) pourraient être ainsi résolus <a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3) grâce à $\mathbf{SAT}$ (plus précisément à $\mathbf{3-SAT}$, qui [sert de référence](https://fr.wikipedia.org/wiki/Probl%C3%A8me_3-SAT) pour la [$\mathbf{NP}$-complétude](https://fr.wikipedia.org/wiki/Probl%C3%A8me_NP-complet))





<a name="cite_note-3"></a>3. [^](#cite_ref-3) les problèmes polynomiaux peuvent aussi être résolu par $\mathbf{SAT}$, ce n'est cependant pas pertinent pusiqu'ils sont justement polynomiaux tandis que $\mathbf{SAT}$ peut être exponentiel (sauf si $\mathbf{P} = \mathbf{NP})$

*********


En raison de cette importance, de nombreuses personnes cherchent à optimiser la résolution de formule, en créant des [SAT-Solver](https://en.wikipedia.org/wiki/SAT_solver) les plus efficaces possibles.

Dans ce TP, nous allons utiliser **pycosat**
https://pypi.org/project/pycosat/

La syntaxe de pycosat est légérement différente de la notre, comme le montre l'exemple suivant :


In [59]:
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]
pycosat.solve(cnf)

[1, -2, -3, -4, 5]

`[1, -2, -3, -4, 5]` est une solution possible trouvée par pycosat. Elle équivaut à un $[1,0,0,0,1]$ avec notre notation.

Ce qui signifie 
$x_1  = \top ; x_2 = x_3 = x_4 = \bot $ et $x_5 = \top$

Dans notre formule

$(x_1 \vee \neg x_5 \vee x_4) \wedge (\neg x_1 \vee  x_5 \vee x_3 \vee x_4) \wedge (\neg x_3 \vee \neg x_4)$ 

on a donc 

$(\top \vee \neg \top \vee \bot) \wedge (\neg \top \vee  \top \vee \bot\vee \bot) \wedge (\neg \bot \vee \neg \bot)$ 

soit 
\begin{align*}
(\top \vee \bot \vee \bot) \wedge ( \bot  \vee  \top \vee \bot\vee \bot) \wedge (\top \vee \top) \\
= \top \wedge \top \wedge \top \\
= \top
\end{align*}

La solution est donc valide.
Cependant on ne retrouve pas notre solution brute-force
$x_1  = x_2 = x_3 = x_4 = x_5 = \bot $.

**Pycosat** nous propose toutefois d'énumérer toutes les solutions :

In [19]:
for sol in pycosat.itersolve(cnf):
    print(sol)

[1, -2, -3, -4, 5]
[1, -2, -3, 4, -5]
[1, -2, -3, 4, 5]
[1, -2, 3, -4, -5]
[1, -2, 3, -4, 5]
[1, 2, 3, -4, -5]
[1, 2, 3, -4, 5]
[1, 2, -3, -4, 5]
[1, 2, -3, 4, -5]
[1, 2, -3, 4, 5]
[-1, 2, -3, 4, -5]
[-1, 2, -3, 4, 5]
[-1, 2, -3, -4, -5]
[-1, 2, 3, -4, -5]
[-1, -2, 3, -4, -5]
[-1, -2, -3, -4, -5]
[-1, -2, -3, 4, 5]
[-1, -2, -3, 4, -5]


On retrouve bien $x_1  = x_2 = x_3 = x_4 = x_5 = \bot $ qui correspond à <tt>[-1,-2,-3,-4,-5] </tt> ($[0,0,0,0,0]$ avec notre syntaxe)

*********




# Sudoku -- Shidoku

On va maintenant essayer de résoudre un problème "plus concret".

Le problème général (sur une grille $n\times n$) du sudoku est $\mathbf{NP}$-complet, on ne connait pas d'algorithme efficace pour le résoudre.
Une approche classique de résolution est le _backtracking_, nous allons de nôtre côté utiliser pycosat.

Pour simplifier le TP, on commencera par la version $4 \times 4$ du sudoku (aussi appelé shidoku)

On se donne quelques grilles d'exemple (vous pouvez créer les vôtres).

In [20]:

# Exemple de grille de Shidoku (0 représente une case vide)
# https://masteringsudoku.com/shidoku/
easy = [
    [ 0, 0, 0, 0],
    [ 3, 0, 2, 0],
    [ 0, 0, 0, 0],
    [ 4, 0, 0, 1],
]
medium = [
    [ 1, 0, 0, 0],
    [ 0, 2, 0, 0],
    [ 0, 0, 3, 0],
    [ 0, 0, 0, 4],
]

hard =[
    [ 0, 0, 0, 0],
    [ 0, 3, 2, 0],
    [ 0, 1, 4, 0],
    [ 0, 0, 0, 0],
]
# Exemple de grille de Shidoku (0 représente une case vide)
blank = [
    [ 0, 0, 0, 0],
    [ 0, 0, 0, 0],
    [ 0, 0, 0, 0],
    [ 0, 0, 0, 0],
]

def show_board(board):
    for row in board:
        print(row)
        
        
show_board(easy)

[0, 0, 0, 0]
[3, 0, 2, 0]
[0, 0, 0, 0]
[4, 0, 0, 1]


On va encoder une grille de la manière suivante : 

Dans ce problème, les variable $v_{(i,j,d)}$ signifie "la cellule ligne $i$ colonne $j$ contient le chiffre $d$".

Ainsi, la variable $v_{(1,3,2)}$ signifie "La cellule à la ligne 1 et colonne 3 contient un 2"

et $\neg v_{(1,3,2)}$ signifie évidemment "La cellule à la ligne 1 et colonne 3 ne contient pas un 2"

Malheureusement c'est une représentation $3$D (une variable est désignée par 3 nombres i, j, d), et nos CNF sont $1$D (un nombre désigne une variable). La fonction `x(i,j,d)` permet donc de faire une transformation 3D vers 1D

$v_{(i,j,d)} \leadsto v_{16*(i-1) + 4*(j-1) + d}$

Par exemple $v_{2,1,3}$ signifie que la case située en ligne $2$, colonne $1$ contient le chiffre $3$ (comme c'est le cas dans la grille <tt>easy</tt>). Cela se traduit par la variable $x_{16*(i-1) + 4*(j-1) + d} = x_{16*(2-1) + 4*(1-1) + d} = x_{19}$.

On obtient ainsi une réprésentation unique pour le placement de chaque chiffre dans chaque case.

Evidemment, vous utiliserez la notation $v(i,j,d)$, beaucoup plus lisible.




In [21]:
#variable "the cell i,j contains digit d"
#example:  19 <-> v_{16*(2-1) + 4*(1-1) + 3} means the cell at row 2 and column 1 contains a 3

def v(i, j, d): 
    return 16 * (i - 1) + 4 * (j - 1) + d



Les fonctions suivantes permettent de transformer la solution du SAT-solver en chiffres à ajouter dans le shidoku.

In [23]:
def read_cell(sol,i, j):
    # return the digit of cell i, j according to the solution
    for d in range(1, 5):
        if v(i, j, d) in sol: #variable found -> v_{i,j,d} = True. Otherwise -v_{i,j,d} would be found
            return d

#puis on remplit la grille        
        
def fill_board(sol):
    grid=blank    
    for i in range(1, 5):
        for j in range(1, 5):
            grid[i - 1][j - 1] = read_cell(sol,i, j)
    return grid

**_question 7_**

On donne une fonction <tt> clause </tt> qui interdit aux cases stockée dans <tt>cells</tt> de contenir le même chiffre

Utiliser cette fonction pour créer une fonction <tt>shidoku_clauses()</tt> correspondant aux règles du Shidoku.

In [29]:
        
def clause_forbidSameValues(res,cells):
    #verify that each cell in res has a different colour
    for i, xi in enumerate(cells):
        for j, xj in enumerate(cells):
            if i < j:
                for d in range(1, 5):
                    res.append([-v(xi[0], xi[1], d), -v(xj[0], xj[1], d)])
    return res
#exemple
clause_forbidSameValues([],[(1,1),(2,3),(4,1)])
#les cases (1,1) (2,3) et (4,1) n'ont pas le droit de contenir pas le même chiffre

[[-1, -25],
 [-2, -26],
 [-3, -27],
 [-4, -28],
 [-1, -49],
 [-2, -50],
 [-3, -51],
 [-4, -52],
 [-25, -49],
 [-26, -50],
 [-27, -51],
 [-28, -52]]

In [28]:
#règles globales du shidoku
def shidoku_clauses(): 
    res = []
    # for all cells, ensure that each cell contains exactly one digit:


   
    # ensure rows and columns have distinct values

    # ensure 2x2 sub-grids "regions" have distinct values
    
    
    return res

#ajoute les contraintes de la grille préremplie
def gridToCNF(grid=blank):
    #turns a grid into a CNF
    clauses = shidoku_clauses()
    
    return clauses

**_question 8_**

Utiliser **pycosat** pour résoudre le shidoku.

In [30]:
def solve(grid=blank):


    return grid


show_board(easy)
show_board(solve(easy))

[0, 0, 0, 0]
[3, 0, 2, 0]
[0, 0, 0, 0]
[4, 0, 0, 1]
P CNF 500(number of clauses)
Time: 0.0
[2, 4, 1, 3]
[3, 1, 2, 4]
[1, 3, 4, 2]
[4, 2, 3, 1]
